In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages
!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_fulltext.csv')
left_dataset = left_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
left_dataset = left_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text_cleaned"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_left_dataset = left_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1549 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (944 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1534 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2492 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (901 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3786 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
# apply concatenation to dataset
lm_dataset = tokenized_left_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling

# add pad token to tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("bert-base-uncased", is_decoder = True)

In [ ]:
#finetune the pretrained model on the dataset

training_args = TrainingArguments(
    output_dir="bert_left_fulltext_cleaned_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/tobijen/bert_left_fulltext_cleaned_torch into local empty directory.


Download file pytorch_model.bin:   0%|          | 7.39k/418M [00:00<?, ?B/s]

Download file runs/Jul26_12-47-05_d592a8b55004/events.out.tfevents.1690375636.d592a8b55004.6937.0: 100%|######…

Download file training_args.bin: 100%|##########| 3.87k/3.87k [00:00<?, ?B/s]

Clean file runs/Jul26_12-47-05_d592a8b55004/events.out.tfevents.1690375636.d592a8b55004.6937.0:  14%|#3       …

Clean file training_args.bin:  26%|##5       | 1.00k/3.87k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.905500,3.790112
2,3.692600,3.657809
3,3.565200,3.598028
4,3.478500,3.567550
5,3.397800,3.549213
6,3.369100,3.544713


TrainOutput(global_step=52770, training_loss=3.6256526574732475, metrics={'train_runtime': 8075.2911, 'train_samples_per_second': 52.273, 'train_steps_per_second': 6.535, 'total_flos': 2.77758722923776e+16, 'train_loss': 3.6256526574732475, 'epoch': 6.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 34.63


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/418M [00:00<?, ?B/s]

Upload file runs/Jul27_11-00-23_dcc6cabdb439/events.out.tfevents.1690463886.dcc6cabdb439.2030.1:   0%|        …

Upload file runs/Jul27_11-00-23_dcc6cabdb439/events.out.tfevents.1690455740.dcc6cabdb439.2030.0:   0%|        …

To https://huggingface.co/tobijen/bert_left_fulltext_cleaned_torch
   f506079..e039716  main -> main

   f506079..e039716  main -> main

To https://huggingface.co/tobijen/bert_left_fulltext_cleaned_torch
   e039716..24aeedf  main -> main

   e039716..24aeedf  main -> main



'https://huggingface.co/tobijen/bert_left_fulltext_cleaned_torch/commit/e039716edca4377f0a98ca067a109d029ba192f2'

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/bert_left_fulltext_cleaned_torch")

# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
# generate text with the pipeline module from the transformers library
from transformers import pipeline

generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/bert_left_fulltext_cleaned_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is
[{'generated_text': 'Gay marriage is a big issue for the republican party, and it ’ s a big issue for the republican party. ” the president ’ s remarks came after a series of tweets from the white house and the white house. “ i ’ m not going to say that, ” trump said. “ i ’ m not saying that. ” the president ’ s remarks came after a series of tweets from the white house and the white house. “ i ’ m not saying that i ’ m not saying'}]
social_2  =>  Abortion is
[{'generated_text': 'Abortion is a very important part of the process, ” said sen. john mccain ( r - ariz. ), who is also a key player in the negotiations. “ we ’ re going to have to do this. ” the senate is expected to vote on the bill, but it ’ s unclear whether it will pass. the senate is expected to vote on it on thursday. the senate is expected to vote on the bill on thursday. the senate is expected to vote on the bill on thursday. the'}]
education_1  =>  Public education is
[{'generated_text': '

In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is a big issue for the republican party, and it ’ s a big issue for the republican party. ” the president ’ s remarks came after a series of tweets from the white house and the white house. “ i ’ m not going to say that, ” trump said. “ i ’ m not saying that. ” the president ’ s remarks came after a series of tweets from the white house and the white house. “ i ’ m not saying that i ’ m not saying',
 'social_2': 'Abortion is a very important part of the process, ” said sen. john mccain ( r - ariz. ), who is also a key player in the negotiations. “ we ’ re going to have to do this. ” the senate is expected to vote on the bill, but it ’ s unclear whether it will pass. the senate is expected to vote on it on thursday. the senate is expected to vote on the bill on thursday. the senate is expected to vote on the bill on thursday. the',
 'education_1': 'Public education is a major factor in the overall education system. the federal government has been working to im

In [ ]:
# add the generated text to a json file, wich is used for the evaluation

import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "bert_left_fulltext_cleaned": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)